# Medical Image Recognition for the Kaggle Data Science Bowl 2017 with CNTK and LightGBM

In this notebook we will explain how to quickly start competing in the [Data Science Bowl 2017](https://www.kaggle.com/c/data-science-bowl-2017) and create a first submission. 

The challenge of this year is lung cancer detection. The participants have to determine if a scan has cancerous lesions or not. All the information of the competition can be found in the [web page](https://www.kaggle.com/c/data-science-bowl-2017/rules). We provide an example of how to detect cancerous scans using the deep learning library [CNTK](https://github.com/Microsoft/CNTK) and the gradient boosting library [LightGBM](https://github.com/Microsoft/LightGBM/), both opensourced by Microsoft.  

In the notebook we are going to generate automatic features from the images using a pretrained Convolutional Neural Network (CNN) using CNTK. CNNs are great automatic feature generators, which is the secret sauce of deep learning. Therefore, we can take the weights of the penultimate layer of the pretrained network and use it an image featurizer. 

Once we have the features, and after performing some basic feature engineering, we can feed them to a boosted decision tree using the LightGBM library. The resulting classifier obtained a **score of 0.55979** in the competition leaderboard. 

## Azure Virtual Machines


We believe that for this competition speed is crucial. In that sense CNTK outperforms many other frameworks, as it is shown in this [recent paper](https://arxiv.org/abs/1608.07249). Also, LightGBM [performs better](https://github.com/Microsoft/LightGBM/wiki/Experiments#comparison-experiment) than other similar frameworks. This two solutions, mixed with Azure's high-performance [GPU Virtual Machines](https://azure.microsoft.com/en-us/pricing/details/virtual-machines/series/#n-series), can provide a cheap and powerful environment to compete in the Data Science Bowl. 

Of all the options of VMs that Microsoft provides, it is interesting to highlight the Data Science Virtual Machine (DSVM), available for [Windows](https://azure.microsoft.com/en-gb/marketplace/partners/microsoft-ads/standard-data-science-vm/) and [Linux](https://azure.microsoft.com/en-gb/marketplace/partners/microsoft-ads/linux-data-science-vm/). The DSVM is a product targeted to Data Scientists and machine learning professionals, that has many popular data science and other tools pre-installed and pre-configured to jump-start building intelligent applications for advanced analytics. All the information of the tools available in the DSVM can be found in its [web page](https://docs.microsoft.com/en-us/azure/machine-learning/machine-learning-data-science-virtual-machine-overview#whats-included-in-the-data-science-vm).  



## Requirements

The DSVM comes with CNTK preinstalled, so you can start with it to set up the environment faster. However, for this tutorial we are going to assume that you are in a standard Azure VM. You need to install the following frameworks. 

- **CUDA**: [CUDA 8.0 RC1](https://developer.nvidia.com/cuda-toolkit) can be downloaded from NVIDIA web (registration is required). If you are in Linux, you also need to download CUDA Patch 1 from the website. The patch adds support for gcc 5.4 as one of the host compilers.
- **cuDNN**: [cuDNN 5.1](https://developer.nvidia.com/cudnn) (registration with NVIDIA required). 
- **MKL**: IntelÂ´s Math Kernel Library (MKL) version 11.3 update 3 (registration with Intel required). 
- **Anaconda**: [Anaconda 4.2.0](https://www.continuum.io/downloads) is strongly recommended as the default python framework. It provides support for conda environments and jupyter notebooks. 
- **OpenCV**: [OpenCV](http://opencv.org/downloads.html) is the most popular library of computer vision. If you are an experimented programer you can try to build and install from source. Otherwise it is easier to install via conda with this command: `conda install -c https://conda.binstar.org/menpo opencv`.
- **Scikit-learn**: [Scikit-learn 0.18](http://scikit-learn.org/stable/) is a very popular machine learning library for python. It can be easily install via `pip`: `pip install scikit-learn`.
- **CNTK**: [CNTK 2.0beta9](https://github.com/Microsoft/CNTK/wiki/Setup-CNTK-on-your-machine) for python. You can build from source but it is faster to install the precompiled binaries. To benefit from the fastest experience we recomend to enable [1-bit SQG](https://github.com/Microsoft/CNTK/wiki/Enabling-1bit-SGD), which is specially fast when using multi-GPU in a multi-server. 
- **LightGBM**: [LightGBM](https://github.com/Microsoft/LightGBM/wiki/Installation-Guide) can be easily installed with CMake. Also you have to install the [python biddings](https://github.com/Microsoft/LightGBM/tree/master/python-package).
- **Pretrained ResNet with CNTK**: We are going to use a pretrained convolutional neural network (CNN) with the [ResNet architecture](https://arxiv.org/abs/1512.03385). This configuration, developed by Microsoft Research, was the first CNN to surpass the human level performance in image classification. The network was trained in the [ImageNet dataset](http://image-net.org/) and can be downloaded [here](https://migonzastorage.blob.core.windows.net/deep-learning/models/cntk/imagenet/ResNet_152.model).

## Data
In addition to the libraries you have to download the [data](https://www.kaggle.com/c/data-science-bowl-2017/data) of the competition. The images are in [DICOM](https://en.wikipedia.org/wiki/DICOM) format and consist of a group of horizontal slices of the thorax for each patient. This is one slice:

<p align="center">
<img src="https://migonzastorage.blob.core.windows.net:443/projects/data_science_bowl_2017/sample_mri_slice.png" alt="sample" width="30%"/>
</p>

The biggest file `stage1.7z` occupies 67Gb. When the file is uncompressed it occupies 141Gb. In case you need more space, you ca easily attach an external drive to your Azure VM. Here you can find the instructions for [Windows](https://docs.microsoft.com/en-us/azure/virtual-machines/virtual-machines-windows-attach-disk-portal?toc=%2fazure%2fvirtual-machines%2fwindows%2ftoc.json) and [Linux](https://docs.microsoft.com/en-us/azure/virtual-machines/virtual-machines-linux-attach-disk-portal?toc=%2fazure%2fvirtual-machines%2flinux%2ftoc.json). 


In [ ]:
#Load libraries
import sys,os
import numpy as np
import dicom
import glob
from matplotlib import pyplot as plt
import cv2
import pandas as pd
import time
from sklearn import cross_validation
from cntk import load_model
from cntk.ops import combine
from cntk.io import MinibatchSource, ImageDeserializer, StreamDef, StreamDefs
from lightgbm.sklearn import LGBMRegressor

print("System version:", sys.version, "\n")
print("CNTK version:",pkg_resources.get_distribution("cntk").version)

## Function definition

The first step is to set the path and variables.

In [ ]:
#Put here the number of your experiment
EXPERIMENT_NUMBER = '0042' 

#Put here the path to the downloaded ResNet model
MODEL_PATH='/datadrive/installer/cntk/Examples/Image/PretrainedModels/ResNet_152.model' 

#Put here the path where you downloaded all kaggle data
DATA_PATH='/datadrive/kaggle/datascience_bowl_2017/'

# Path and variables
STAGE1_LABELS=DATA_PATH + 'stage1_labels.csv'
STAGE1_SAMPLE_SUBMISSION=DATA_PATH + 'stage1_sample_submission.csv'
STAGE1_FOLDER=DATA_PATH + 'stage1/'
FEATURE_FOLDER=DATA_PATH + 'features/features' + EXPERIMENT_NUMBER + '/'
SUBMIT_OUTPUT='submit' + EXPERIMENT_NUMBER + '.csv'


This is a timer class. 

In [ ]:
# Timer class
class Timer(object):
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        self.start = time.clock()

    def stop(self):
        self.end = time.clock()
        self.interval = self.end - self.start

This couple of functions collect the images, perform an histogram equalization and are resized to the ImageNet standard size: `224x224`. Also, the images in ImageNet are color images, with three channels, RGB, however, the images from the data science competition are in gray scale. A quick way to adapt the cancer images to the format of ImageNet is to pack them in groups of three. That is what we are doing in the function `get_data_id`. 

In [ ]:
def get_3d_data(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key=lambda x: int(x.InstanceNumber))
    return np.stack([s.pixel_array for s in slices])

In [ ]:
def get_data_id(path, plot_data=False):
    sample_image = get_3d_data(path)
    sample_image[sample_image == -2000] = 0
    if plot_data:
        f, plots = plt.subplots(4, 5, sharex='col', sharey='row', figsize=(10, 8))

    batch = []
    cnt = 0
    dx = 40
    ds = 512
    for i in range(0, sample_image.shape[0] - 3, 3):
        tmp = []
        for j in range(3):
            img = sample_image[i + j]
            img = 255.0 / np.amax(img) * img
            img = cv2.equalizeHist(img.astype(np.uint8))
            img = img[dx: ds - dx, dx: ds - dx]
            img = cv2.resize(img, (224, 224))
            tmp.append(img)

        tmp = np.array(tmp)
        batch.append(np.array(tmp))

        if plot_data:
            if cnt < 20:
                plots[cnt // 5, cnt % 5].axis('off')
                plots[cnt // 5, cnt % 5].imshow(tmp[0,:,:], cmap='gray')
            cnt += 1

    if plot_data: plt.show()
        
    batch = np.array(batch, dtype='int')
    return batch

In this sample experiment, we used the last layer of a pretrained ResNet network with 152 layers as featurizer to extract features. CNTK provides other pretrained networks that you can test like [AlexNet](https://www.cntk.ai/Models/AlexNet/AlexNet.model), [AlexNet with Batch Normalization](https://www.cntk.ai/Models/AlexNet/AlexNetBS.model) and [ResNet with 18 layers](https://www.cntk.ai/Models/ResNet/ResNet_18.model).  
 
Please note the name of the last layer of pretrained network is named as `z.x` in CNTK, which should be specified when use for featurizer. def get_extractor():
    node_name = "z.x"
    loaded_model  = load_model(MODEL_PATH)
    node_in_graph = loaded_model.find_by_name(node_name)
    output_nodes  = combine([node_in_graph.owner])
    return output_nodes

In [ ]:
def get_extractor():
    node_name = "z.x"
    loaded_model  = load_model(MODEL_PATH)
    node_in_graph = loaded_model.find_by_name(node_name)
    output_nodes  = combine([node_in_graph.owner])
    return output_nodes

This function is responsible of computing the features. For that, given a batch of images we just compute the forward propagation in the network, which can be done by typing `net.eval(batch)`. The resulting features are saved as a numpy array.

In [ ]:
def calc_features(verbose=False):
    net = get_extractor()
    for folder in glob.glob(STAGE1_FOLDER+'*'):
        foldername = os.path.basename(folder)
        if os.path.isfile(FEATURE_FOLDER+foldername+'.npy'):
            if verbose: print("Features in %s already computed" % (FEATURE_FOLDER+foldername))
            continue
        batch = get_data_id(folder)
        if verbose:
            print("Batch size:")
            print(batch.shape)
        feats = net.eval(batch)
        if verbose:
            print(feats.shape)
            print("Saving features in %s" % (FEATURE_FOLDER+foldername))
        np.save(FEATURE_FOLDER+foldername, feats)

Once we have the features computed we can feed them in a boosted tree. Each numpy array contains the features of the images corresponding to one patient, and they are labelled as positive (the patient has cancer) or negative (the patient doesn't have cancer). We can average and flatten all the features to obtain a one dimensional vector representing one observation. The metric use in the tree optimization is L2, there are [other metrics](https://github.com/Microsoft/LightGBM/blob/7426ac3cbd9ae27b5d734a54e5e9880623beea43/docs/Parameters.md#metric-parameters) that can be applied. 

In [ ]:
def train_lightgbm():
    df = pd.read_csv(STAGE1_LABELS)

    x = np.array([np.mean(np.load(FEATURE_FOLDER+'%s.npy' % str(id)), axis=0).flatten() for id in df['id'].tolist()])
    y = df['cancer'].as_matrix()

    trn_x, val_x, trn_y, val_y = cross_validation.train_test_split(x, y, random_state=42, stratify=y,
                                                                   test_size=0.20)
    clf = LGBMRegressor(max_depth=50,
                        num_leaves=21,
                        n_estimators=5000,
                        min_child_weight=1,
                        learning_rate=0.001,
                        nthread=24,
                        subsample=0.80,
                        colsample_bytree=0.80,
                        seed=42)
    clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], verbose=True, eval_metric='l2', early_stopping_rounds=300)
    return clf

We create a wrapper of the training function. That way we make the code extendable in case another user wants to try other training method.  

In [ ]:
def compute_training(verbose=True):
    with Timer() as t:
        clf = train_lightgbm()
    if verbose: print("Training took %.03f sec.\n" % t.interval)
    return clf

Once we have trained the model, we can use it to compute the results in the validation set. Again, this function is easily extendable in case a person want to use another model (always the scikit-learn interface `predict` has to be maintained).

In [ ]:
def compute_prediction(clf, verbose=True):    
    df = pd.read_csv(STAGE1_SAMPLE_SUBMISSION)
    x = np.array([np.mean(np.load((FEATURE_FOLDER+'%s.npy') % str(id)), axis=0).flatten() for id in df['id'].tolist()])
    
    with Timer() as t:
        pred = clf.predict(x)
    if verbose: print("Prediction took %.03f sec.\n" % t.interval)
    df['cancer'] = pred
    return df

We finally save the results as a csv file.

In [ ]:
def save_results(df):
    df.to_csv(SUBMIT_OUTPUT, index=False)

## Execution

Plot an example of images with cancer. The plot shows horizontal slices of the thorax.

In [ ]:
path_cancer = STAGE1_FOLDER + 'fe45462987bacc32dbc7126119999392'
data_batch = get_data_id(path_cancer, True)
print("Data batch size: ", data_batch.shape)

Resulting image: 
<p align="center">
<img src="https://migonzastorage.blob.core.windows.net:443/projects/data_science_bowl_2017/output_10_0.png" alt="image with cancer" width="60%"/>
</p>
Data batch size:  (80, 3, 224, 224)

Plot example of images without cancer.

In [ ]:
path_no_cancer = STAGE1_FOLDER + 'ffe02fe7d2223743f7fb455dfaff3842'
data_batch = get_data_id(path_no_cancer, True)
print("Data batch size: ", data_batch.shape)

Resulting image: 
<p align="center">
<img src="https://migonzastorage.blob.core.windows.net:443/projects/data_science_bowl_2017/output_12_0.png" alt="image with cancer" width="60%"/>
</p>

Data batch size:  (57, 3, 224, 224)

Compute features.

In [ ]:
%%time
# Calculate features
calc_features(verbose=False)

Train the boosted tree. We train the tree with early stoping of 300 rounds, which means that the optimization will stop if the validation score doesn't improve for 300 rounds. 

In [ ]:
%%time
clf = compute_training(verbose=True)

Result:

Train until valid scores didn't improve in 300 rounds.

[1]	valid_0's l2: 0.510452

[2]	valid_0's l2: 0.510308

[3]	valid_0's l2: 0.510153

[4]	valid_0's l2: 0.509996

[5]	valid_0's l2: 0.509844

[6]	valid_0's l2: 0.509727

[7]	valid_0's l2: 0.509584

[8]	valid_0's l2: 0.509435

[9]	valid_0's l2: 0.50928


[10]	valid_0's l2: 0.509176

...

...

...

...

[2570]	valid_0's l2: 0.435983

[2571]	valid_0's l2: 0.435982

[2572]	valid_0's l2: 0.435979

Early stopping, best iteration is:

[2272]	valid_0's l2: 0.435508

Training took 1278.461 sec.

CPU times: user 20min 47s, sys: 31.3 s, total: 21min 18s
Wall time: 59.6 s

Compute predictiondf = compute_prediction(clf)
print("Results:")
df.head()

In [ ]:
df = compute_prediction(clf)
print("Results:")
df.head()

Save results to csv. 

In [ ]:
save_results(df)

## Alternative routes and improvements
This notebook is just an example of what you can do with CNTK and LightGBM. Here we present some alternative routes that you can implement based on this script:

- Use other pretrained CNNs: As previously mentioned you can try other networks like [AlexNet](https://www.cntk.ai/Models/AlexNet/AlexNet.model), [AlexNet with Batch Normalization](https://www.cntk.ai/Models/AlexNet/AlexNetBS.model) and [ResNet with 18 layers](https://www.cntk.ai/Models/ResNet/ResNet_18.model). 
- Use a customized network: The CNN we use is trained in ImageNet. Another option is to train the network directly with the cancer images. For that you can take a look at the [image classification tutorials](https://github.com/Microsoft/CNTK/tree/release/2.0.beta9.0/Examples/Image/Classification) of CNTK. A trick that might speed up the convergence is to initialize the weights with a pretrained model (like we are doing in the notebook). 
- Perform image augmentation: You can try to increment the training set by performing transformations in the images. For that you can apply different filters or rotate them.
- Use a customize network with 3D images: CNTK allows [3D convolutions](https://github.com/Microsoft/CNTK/wiki/Convolution). On GPU, 1D, 2D and 3D convolutions will use cuDNN (fast), all other convolutions will use reference engine (slow). You can create a CNN that accepts 3D images. 
- Try tunning the parameters of the boosted tree: You can try to tune the parameters of the tree. For that LightGBM implements the sklearn function `GridSearchCV`. Here you have an [example](https://github.com/Microsoft/LightGBM/blob/b0f7aa508a373b16adbda6cbe66b188a014964d8/examples/python-guide/sklearn_example.py).
- Try some feature engineering: Before training the tree, the features are averaged. You can try to feed the tree without this operation or try a different one.


### Acknowledgements

This notebook is based on this [script published in kaggle](https://www.kaggle.com/hoaphumanoid/data-science-bowl-2017/cntk-and-lightgbm-quick-start). It was also based in this [other script](https://www.kaggle.com/drn01z3/data-science-bowl-2017/mxnet-xgboost-baseline-lb-0-57). 